In [3]:
import random
import pandas as pd


"Hello world"

'Hello world'

# Data Models


In [43]:
import sqlite3

conn = sqlite3.connect(':memory:')

print ("Opened database successfully")

conn.execute('''CREATE TABLE IF NOT EXISTS STUDENT
         (
         NAME           TEXT    NOT NULL
         );''')


conn.execute('''CREATE TABLE IF NOT EXISTS COURSE
         (
         COURSE_CODE CHAR(8) UNIQUE NOT NULL,
         NAME           TEXT    NOT NULL
         );''')

conn.execute('''CREATE TABLE IF NOT EXISTS STUDENT_COURSE
         (
         STUDENT_ID INT,
         COURSE_ID INT,
         FOREIGN KEY(STUDENT_ID) REFERENCES STUDENT(ID),
         FOREIGN KEY(COURSE_ID) REFERENCES COURSE(ID)
         );''')


conn.execute('''CREATE TABLE IF NOT EXISTS TEACHER
         (
         NAME           TEXT    NOT NULL
         );''')

conn.execute('''CREATE TABLE IF NOT EXISTS CLASSROOM
         (
         NAME           TEXT    NOT NULL
         );''')

conn.execute('''CREATE TABLE IF NOT EXISTS DAY
         (
         NAME           TEXT    NOT NULL
         );''')





print ("Tables created successfully");


# =======================  INSERT STUDENTS INTO TABLE
students_file = pd.read_csv('studentNames.csv', header=None)
print(students_file)

for index, row in students_file.iterrows():
    student_name = row[0]
    conn.execute(f"INSERT INTO STUDENT (NAME) \
          VALUES ('{student_name}')")

conn.commit()
print ("Records created successfully")


cursor = conn.execute("SELECT rowid, name from STUDENT")
for row in cursor:
   print (row[0], "\t", row[1])

print ("Operation done successfully");



# =======================  INSERT COURSES INTO TABLE
courses_file = pd.read_csv('courses.csv', header=None)
print(courses_file)






## Construct a valid input here (according to what we need)

# Structure (Bottom-Up):
#        1. Gene : Contains 1 course, 1 instructor , and an array of Students, It's corresponding Day (mon, tue, wed ...) ,
#                  It's corresponding Time (9-12, 10-1 ...).
#        2. Chromosome: Contains a classroom , TimeTable for the classroom , fitness value of the generated Timetable
#        3. Member: Contains a list of chromosomes
#        4. Population:  Contains a list of members


# initialise population 



student_course_file = pd.read_csv('studentCourse.csv')
print(student_course_file)



Classrooms_Array = ['c301','c302','c303','c304','c305','c306','c307','c308','c309','c310']
Courses_Array = ['CS217','CS218','CS219','CS220','CS221','CS222']
Instructors_Array = ['Ayesha Bano','Aqeel Shahzad','Usman Rashid','Maimoona Rassol','Muhammad bin Qasim','Sanaa Ilyas']
Students_Array = ['Bano','Aqeel Usman ','Shahzad Rashid','Maimoona Qasim','Muhammad bin Aqeel','Sanaa Rassol']
Days_Array = ['Sunday', 'Monday','Tuesday','Wednesday','Thursday','Friday','Saturday']

class Course:
    def __init__ (self, id, name):
        self.id = id
        self.name = name

class Instructor:
    def __init__ (self, name):
        self.name = name
        
class Student:
    def __init__ (self, name):
        self.name = name

class Classroom:
    def __init__ (self, id, capacity):
        self.id = id
        self.capacity = capacity
        
class Student_Course:
    def __init__ (self, course, student):
        self.course = course
        self.student_id = student


Opened database successfully
Tables created successfully
                     0
0        Mohammed Azam
1    Mohamed F Bakaram
2      Manal Aldaihani
3        Ibrar Manawer
4         Adam Flowers
..                 ...
195     Naveed Hussain
196      Sarah L Clark
197       Maria Lytras
198        Bibi A Khan
199       Iram Matloob

[200 rows x 1 columns]
Records created successfully
1 	 Mohammed Azam
2 	 Mohamed F Bakaram
3 	 Manal Aldaihani
4 	 Ibrar Manawer
5 	 Adam Flowers
6 	 Lilia Kyuchukova
7 	 Sarina Patel
8 	 Aleena Unadkat
9 	 Ibrahim Adegboye
10 	 Salma Khanam
11 	 Kiran K Chahal
12 	 Omar F Khan
13 	 Seema Mandalia
14 	 Asim Ali
15 	 Yasmin Ahmed
16 	 Sheila Hughton
17 	 Sarah Taylor
18 	 Anja Sarlah
19 	 Syed M Abbas
20 	 Hanna I Siahaan
21 	 Sarah Austin
22 	 Mohammed Riyaz Shahul Hameed Mohammed Yakub
23 	 Halima Sayed
24 	 Sarah L Barber-Jones
25 	 Meryem Mehmetbasoglu
26 	 Adam L Finch
27 	 Sarah Hanley
28 	 Sarah Fogg
29 	 Tara Glickman
30 	 Reem N Hassan
31 	 Amir Ahm

# Population Model

In [398]:
class Gene:
    def __init__(
            self,
            course=None,
            instructor=None,
            students=None,
            day=None,
            start_time=None,
            end_time=None,
    ):

        if course is None:
            course = Courses_Array[random.randint(0, len(Courses_Array) - 1)]

        if instructor is None:
            instructor = Instructors_Array[random.randint(0, len(Instructors_Array) - 1)]

        if students is None:
            L = random.randint(1, len(Students_Array))
            students = random.sample(Students_Array, L)

        if day is None:
            day = Days_Array[random.randint(0, len(Days_Array) - 1)]

        if start_time is None:
            start_time = random.randint(9.0, 17.0)

        if end_time is None:
            ## Logical way is to start this range from start_time
            end_time = random.randint(9.0, 17.0)

        self.course = course
        self.instructor = instructor
        self.students = students
        self.day = day
        self.start_time = start_time
        self.end_time = end_time

    def __str__(self):
        return (
                self.day             + "\t" +
                str(self.start_time) + "\t" +
                str(self.end_time)   + "\t" +
                self.course          + "\t" +
                self.instructor      + "\t" +
                str(self.students)   + "\t" +
                "."
               )

class Chromosome:

    def __init__(self,
                 genes=None,
                 classroom_name=None,
                 ):

        self.genes = genes

        if self.genes is None:
            self.genes = []
            for i in range(0, random.randint(1, 18)):
                self.genes.append(Gene())

        if classroom_name is None:
            classroom_name = Classrooms_Array[random.randint(0, len(Classrooms_Array) - 1)]
        
        self.classroom_name = classroom_name

        self.index = -1
        
    def __getitem__(self, index):
        return self.genes[index]


class Member:

    def __init__(self,
                 chromosomes=None
                 ):
        self.chromosomes = chromosomes
        self.index = -1
        
        # ToDo: Make this default to zero!
        self.fitness_value = random.randint(0,1000)

        if self.chromosomes is None:
            self.chromosomes = []
            
            ## More generic way is to randomize
            #for i in range (0, len(Classrooms_Array) - 1):
            #   self.chromosomes.append ( Chromosome() )

            ## But this is more logical
            for classroom in Classrooms_Array:
                self.chromosomes.append(Chromosome(classroom_name=classroom))

    def __getitem__(self, index):
        return self.chromosomes[index]


class Population:

    def __init__(self,
                 members=None
                 ):
        self.members = members
        self.index = 0

        if self.members is None:
            self.members = []
            for i in range(1, 20):
                self.members.append(Member())

    def __getitem__(self, index):
        return self.members[index]
    

# Initialization

In [399]:
def initialize_population():
    
    new_population = Population()

    
#     ----------------------------------------------- Debugging
#     for member in new_population.members:
#         for chromosome in member.chromosomes:
#             for gene in chromosome.genes:
#                 print (gene.course,gene.instructor,gene.students,gene.day,gene.start_time,gene.end_time)
#     ----------------------------------------------- Debugging


    return new_population

In [400]:
new_pop = initialize_population()

for member in new_pop:
    for chromosome in member:
        print("\n\nCLASSROOM:", chromosome.classroom_name)
        sort_by_day = sorted(chromosome, key=lambda gene: (gene.day, gene.start_time) )
        for gene in sort_by_day:
            print(gene)



CLASSROOM: c301
Friday	13	17	CS221	Ayesha Bano	['Muhammad bin Aqeel']	.
Friday	14	17	CS221	Ayesha Bano	['Bano']	.
Monday	11	15	CS221	Aqeel Shahzad	['Bano', 'Maimoona Qasim', 'Muhammad bin Aqeel', 'Sanaa Rassol']	.
Thursday	16	16	CS221	Muhammad bin Qasim	['Bano', 'Muhammad bin Aqeel', 'Aqeel Usman ', 'Sanaa Rassol']	.
Tuesday	13	10	CS222	Muhammad bin Qasim	['Maimoona Qasim']	.


CLASSROOM: c302
Friday	9	11	CS217	Usman Rashid	['Aqeel Usman ', 'Bano', 'Sanaa Rassol']	.
Friday	9	11	CS217	Sanaa Ilyas	['Shahzad Rashid']	.
Friday	15	10	CS219	Aqeel Shahzad	['Sanaa Rassol', 'Aqeel Usman ']	.
Friday	15	14	CS221	Muhammad bin Qasim	['Aqeel Usman ', 'Muhammad bin Aqeel', 'Shahzad Rashid', 'Bano', 'Sanaa Rassol']	.
Friday	15	12	CS220	Maimoona Rassol	['Aqeel Usman ', 'Shahzad Rashid']	.
Monday	9	12	CS218	Sanaa Ilyas	['Aqeel Usman ', 'Bano', 'Shahzad Rashid', 'Muhammad bin Aqeel']	.
Monday	13	10	CS222	Muhammad bin Qasim	['Maimoona Qasim', 'Aqeel Usman ']	.
Thursday	11	10	CS221	Ayesha Bano	['Maimoona

# Benchmark / Fitness Stuff

In [401]:
minimum_soft_constraints = 0


def calculate_minimum_benchmark(domain):
    
    pass


def set_fitness(chromosome):
    
    # Check hard constraints (All)
    
    # ...
    
    # Check soft constraints (randomize, set counter, 
    # check if counter==minimum_soft_constraints, if yes then it is   g o o d   e n o u g h)
     
    pass


# Selection Stuff

In [403]:
def roulette_wheel_selection(population):
    
    pass


def elitism(population):
    max_index_1 = 0
    max_index_2 = 0
    max_fitness1 = 0
    max_fitness2 = 0
    #     print(population)

    for idx, individual in enumerate(population.members):
        if individual.fitness_value > max_fitness2:
            if individual.fitness_value > max_fitness1:
                max_fitness2 = max_fitness1
                max_index_2 = max_index_1
                max_fitness1 = individual.fitness_value
                max_index_1 = idx
            elif individual.fitness_value > max_fitness2:
                max_fitness2 = individual.fitness_value
                max_index_2 = idx

    return population[max_index_1], population[max_index_2]



def select_best_mom_and_dad(population):
    
    return elitism(population)

    ## Since we're instructed to do Roulette_wheel_selection...
    #return roulette_wheel_selection(population)

In [404]:
new_pop = initialize_population()

mom,dad = select_best_mom_and_dad(new_pop)

print(" M O M ")
for chromosome in mom:
    for gene in chromosome:
        print(gene)
        
print()
print(" D A D ") 

for chromosome in dad:
    for gene in chromosome:
        print(gene)

 M O M 
Monday	12	17	CS222	Muhammad bin Qasim	['Aqeel Usman ', 'Bano', 'Maimoona Qasim']	.
Friday	9	14	CS222	Maimoona Rassol	['Muhammad bin Aqeel', 'Bano', 'Sanaa Rassol', 'Maimoona Qasim', 'Shahzad Rashid']	.
Monday	9	10	CS217	Usman Rashid	['Muhammad bin Aqeel', 'Bano']	.
Monday	11	15	CS217	Ayesha Bano	['Maimoona Qasim', 'Aqeel Usman ', 'Sanaa Rassol', 'Bano', 'Muhammad bin Aqeel', 'Shahzad Rashid']	.
Thursday	15	16	CS221	Maimoona Rassol	['Muhammad bin Aqeel', 'Bano']	.
Friday	14	14	CS217	Ayesha Bano	['Aqeel Usman ', 'Sanaa Rassol', 'Bano', 'Maimoona Qasim', 'Shahzad Rashid']	.
Tuesday	10	13	CS222	Sanaa Ilyas	['Muhammad bin Aqeel', 'Sanaa Rassol', 'Aqeel Usman ', 'Bano', 'Shahzad Rashid']	.
Monday	11	12	CS220	Sanaa Ilyas	['Aqeel Usman ', 'Muhammad bin Aqeel', 'Shahzad Rashid', 'Maimoona Qasim', 'Sanaa Rassol', 'Bano']	.
Monday	10	15	CS222	Muhammad bin Qasim	['Bano', 'Shahzad Rashid']	.
Monday	12	13	CS219	Ayesha Bano	['Shahzad Rashid', 'Muhammad bin Aqeel', 'Sanaa Rassol', 'Maimoona Qa

# Astaghfar Stuff

In [405]:
def crossover(husband, wife):
    length = len(husband.chromosomes)//2
    children = []
    son      = Member( chromosomes = husband.chromosomes[:length] +  wife.chromosomes[length:]    )
    daughter = Member( chromosomes = wife.chromosomes[:length]    +  husband.chromosomes[length:] )
    
    children.append(son)
    children.append(daughter)

    return children

# Zombie Time

In [406]:
def choose_random_possibility_from_domain(domain):
    pass

def choose_random_possibility_from_model(model):
    pass


def mutate(chromosome):
    chromosome[random.randint(0,len(chromosome)-1)] = choose_random_possibility_from_model(model)

# Evaluation


In [407]:
min_soft_constraints = 3

def hard_test_start_time_gt_end_time(data):
    print("Hello hard constraint test")
    return True


def soft_test_consecutive_exams(data):
    print("Hello soft constraint test")
    return True



hard_constraints = [
    {
        "name":"Start_Time > End_Time",
        "func":hard_test_start_time_gt_end_time
    },

]


soft_constraints = [
    {
        "name":"Student should not have consecutive exams",
        "func":soft_test_consecutive_exams
    },
    {
        "name":"Student should not have consecutive exams",
        "func":soft_test_consecutive_exams
    },    
    {
        "name":"Student should not have consecutive exams",
        "func":soft_test_consecutive_exams
    },
    {
        "name":"Student should not have consecutive exams",
        "func":soft_test_consecutive_exams
    },    
]



def evaluate(solution):
    
    for constraint in hard_constraints:
        if (constraint["func"](solution) == False):
            return False
        
    soft_constraint_pass_counter = 0
    for constraint in soft_constraints:
        if (constraint["func"](solution) == True):
            soft_constraint_pass_counter += 1
        if soft_constraint_pass_counter > min_soft_constraints:
            return True
            
    
    return False

In [408]:
evaluate(mom)

Hello hard constraint test
Hello soft constraint test
Hello soft constraint test
Hello soft constraint test
Hello soft constraint test


True

# Our Genetic Algorithm

## Basic
Steps are:
- ~Generate New Population~
- ~Initial Selection of Parents~
- ~Crossover~
- Fitness
- Mutate
- ~Select best mom and dad~
- ~Evaluate~
    

In [417]:
def GeneticAlgorithm():
    
    
#     min_benchmark = calculate_minimum_benchmark(domain)
#     min_distance = random.randint(50, 100)
#     fail_limit = random.randint(0,20)
    
    new_generation = initialize_population()
    
    mom, dad = new_generation[random.randint(0,len(new_generation.members)//2)], new_generation[random.randint(len(new_generation.members)//2,len(new_generation.members)-1)]
    
    
#     while (evaluate(solution)):

    children = crossover (mom, dad)
    print ("\n\n")
    for idx, member in enumerate(children):
        print ("\n\nChild #", idx, ":")
        for chromosome in member:
            for gene in chromosome:
                print (gene)

                
                
                
                
                
    for member in children:
        new_chromosomes = []
        for chromosome in chromosomes:
            set_fitness(chromosome)
            
            if (chromosome.fitness_value < fail_limit):
                continue
            
            while (chromosome.fitness_value < min_distance):
                mutate(chromosome)
                set_fitness(chromosome)     
                    
            new_chromosomes.append( chromosome )

        if len(new_chromosomes) == 0: new_chromosomes = None
        new_generation.members.append( Member(chromosomes = new_chromosomes ) )

  
            
#      mom, dad = select_best_mom_and_dad(new_generation)
        
#      solution = mom.fitness_sum
        
#      print("MOM")

In [418]:
GeneticAlgorithm()






Child # 0 :
Wednesday	15	12	CS222	Sanaa Ilyas	['Maimoona Qasim', 'Muhammad bin Aqeel', 'Aqeel Usman ', 'Shahzad Rashid', 'Sanaa Rassol']	.
Thursday	10	14	CS222	Usman Rashid	['Muhammad bin Aqeel']	.
Friday	12	11	CS218	Sanaa Ilyas	['Bano']	.
Friday	11	15	CS220	Muhammad bin Qasim	['Maimoona Qasim']	.
Monday	10	17	CS218	Maimoona Rassol	['Maimoona Qasim', 'Shahzad Rashid', 'Aqeel Usman ', 'Muhammad bin Aqeel']	.
Wednesday	9	17	CS221	Sanaa Ilyas	['Muhammad bin Aqeel', 'Maimoona Qasim', 'Aqeel Usman ', 'Sanaa Rassol', 'Shahzad Rashid']	.
Wednesday	12	13	CS222	Muhammad bin Qasim	['Muhammad bin Aqeel', 'Maimoona Qasim', 'Sanaa Rassol', 'Bano']	.
Friday	14	12	CS220	Ayesha Bano	['Muhammad bin Aqeel', 'Maimoona Qasim', 'Shahzad Rashid', 'Bano', 'Aqeel Usman ', 'Sanaa Rassol']	.
Monday	11	14	CS222	Aqeel Shahzad	['Shahzad Rashid', 'Sanaa Rassol', 'Bano', 'Muhammad bin Aqeel', 'Aqeel Usman ']	.
Wednesday	15	16	CS222	Usman Rashid	['Muhammad bin Aqeel', 'Shahzad Rashid', 'Bano', 'Maimoona Qasim']	